# A CNN-LSTM framework for the solar wind density forecasting
In this notebook we pre-process solar wind and solar binary mask for the time series analysis


## Notebook Contributors
* Andrea Giuseppe Di Francesco -- email: difrancesco.1836928@studenti.uniroma1.it
* Massimo Coppotelli -- email: coppotelli.1705325@studenti.uniroma1.it

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install torch

In [4]:
import pandas as pd
import numpy as np
import torch


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
proton_density_data = pd.read_csv('./wind-2012-20221014T050711Z-001/wind-2012/WI_H5_SWE_45129.csv', skiprows=list(range(0,275)))

proton_density_data